In [1]:
import networkx as nx
import fastnode2vec

In [4]:
data_path = "/Users/alex/Library/CloudStorage/OneDrive-MUNI/Dokumenty/Projects Data/GNNs/cz_traffic/"
#data_path = "/media/sf_bely/OneDrive - MUNI/Dokumenty/Projects Data/GNNs/cz_traffic/"

In [30]:
G = nx.read_gpickle(data_path + 'czech_road_network.p')
print("G num of nodes = ", G.number_of_nodes(), "num of edges = ", G.number_of_edges())
isolated_nodes = [k for k, v in dict(G.degree()).items() if v == 0]
print("G, isolated nodes:", isolated_nodes)
G.remove_nodes_from(isolated_nodes)
print("G num of nodes = ", G.number_of_nodes(), "num of edges = ", G.number_of_edges())

G num of nodes =  1128 num of edges =  1153
G, isolated nodes: ['D04-012', 'D05-025', 'D35-014', 'I35-141']
G num of nodes =  1124 num of edges =  1153


In [13]:
list(G.edges(data=True))[:10]

[('D00-001', 'D00-002', {}),
 ('D00-001', 'D10-001', {}),
 ('D00-002', 'D00-003', {}),
 ('D00-002', 'D11-001', {}),
 ('D00-003', 'D00-004', {}),
 ('D00-003', 'D11-001', {}),
 ('D00-003', 'I12-001', {}),
 ('D00-004', 'D00-005', {}),
 ('D00-004', 'D01-005', {}),
 ('D00-004', 'D01-004', {})]

In [31]:
G2 = nx.read_gpickle(data_path + 'cz_road_network.p')
print("weighted G num of nodes = ", G2.number_of_nodes(), "num of edges = ", G2.number_of_edges())
isolated_nodes2 = [k for k, v in dict(G2.degree()).items() if v == 0]
print("weighted G, isolated nodes:", isolated_nodes2)
G2.remove_nodes_from(isolated_nodes2)
print("weighted G num of nodes = ", G2.number_of_nodes(), "num of edges = ", G2.number_of_edges())

weighted G num of nodes =  1128 num of edges =  1153
weighted G, isolated nodes: ['D04-012', 'D05-025', 'D35-014', 'I35-141']
weighted G num of nodes =  1124 num of edges =  1153


In [12]:
list(G2.edges(data=True))[:10]

[('D00-001', 'D00-002', {'weight': 1.2050837311556577}),
 ('D00-001', 'D10-001', {'weight': 2.5476607143833463}),
 ('D00-002', 'D00-003', {'weight': 1.2305560445026138}),
 ('D00-002', 'D11-001', {'weight': 4.208540877539873}),
 ('D00-003', 'D00-004', {'weight': 13.864838928754674}),
 ('D00-003', 'D11-001', {'weight': 4.359794366391371}),
 ('D00-003', 'I12-001', {'weight': 8.00865339673411}),
 ('D00-004', 'D00-005', {'weight': 4.745505898863562}),
 ('D00-004', 'D01-005', {'weight': 3.5671360137480663}),
 ('D00-004', 'D01-004', {'weight': 3.2169261012041366})]

In [34]:
def node2vec_embedding(G: nx.Graph, dim=32, walk_length=100, context=32, p=2.0, q=0.5, workers=4, seed=42):
    if nx.is_weighted(G):
        n2v_graph = fastnode2vec.Graph([(str(edge[0]), str(edge[1]), edge[2]['weight']) for edge in G.edges(data=True)],
                directed=False, weighted=True)
    else:
        n2v_graph = fastnode2vec.Graph([(str(edge[0]), str(edge[1])) for edge in G.edges(data=True)],
                    directed=False, weighted=False)
    n2v = fastnode2vec.Node2Vec(n2v_graph, dim=dim, walk_length=walk_length, context=context, p=p, q=q, workers=workers, seed=seed)
    n2v.train(epochs=100)
    n2v_embeddings = [n2v.wv[str(node)] for node in G]
    return n2v_embeddings

In [35]:
embeddings = node2vec_embedding(G)

Training: 100%|██████████| 112400/112400 [00:17<00:00, 6446.89it/s]


In [36]:
# join embeddings with corresponding nodes (as x feature)
for idx, node in enumerate(G.nodes()):
    G.add_node(node, embedding=embeddings[idx])

In [40]:
embeddings2 = node2vec_embedding(G2)
# join embeddings with corresponding nodes (as x feature)
for idx, node in enumerate(G2.nodes()):
    G2.add_node(node, embedding=embeddings2[idx])

Training: 100%|██████████| 112400/112400 [00:17<00:00, 6294.86it/s]


In [42]:
G2.nodes['D00-001']

{'embedding': array([ 0.28925058, -0.5321759 ,  0.1633069 ,  0.22804017,  0.07528   ,
         0.3985399 , -0.76625425,  0.98368603, -0.40598074, -0.7826637 ,
         0.15234624, -0.35742223,  0.5700691 ,  0.10301872,  0.32726422,
         1.1231879 , -0.26717177, -0.7335326 , -0.16624275,  0.11566973,
         0.7801394 ,  1.143941  , -0.44375572, -0.02538498, -0.28431115,
         1.1864569 , -0.22486278, -0.90309584, -0.40372974, -1.263226  ,
         0.64763814, -0.23639019], dtype=float32)}

# Old code follows

In [3]:
def generate_node_embeddings(graph: nx.Graph):
    model = Node2Vec(graph, dimensions=32, num_walks=100, walk_length=80, weight_key='dist', temp_folder='temp/', workers=2)
    result = model.fit(window=5, min_count=1)
    return result

In [5]:
%%time
embeddings = generate_node_embeddings(G)

Generating walks (CPU: 1): 100%|██████████| 50/50 [06:10<00:00,  7.41s/it]


CPU times: user 7min 6s, sys: 53 s, total: 7min 59s
Wall time: 6min 34s


In [7]:
embeddings.wv.most_similar('D00-001')

[('D10-001', 0.935806393623352),
 ('D00-002', 0.8871928453445435),
 ('D10-002', 0.7911334037780762),
 ('D00-003', 0.6844806671142578),
 ('D11-001', 0.683589518070221),
 ('D11-002', 0.6108906269073486),
 ('D10-003', 0.5654127597808838),
 ('I12-001', 0.4950152039527893),
 ('I12-002', 0.46955227851867676),
 ('D11-003', 0.41467419266700745)]

In [9]:
embeddings.save('data/embeddings32.data')

In [ ]:
# join embeddings with corresponding nodes (as x feature)
for idx, node in enumerate(G.nodes()):
    G.add_node(node, embedding=embeddings.wv[str(node)].copy())

In [ ]:
nx.readwrite.write_gpickle(G, 'data/network32.gpickle')